In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Doing translation of a Sample audio .wev file to text**

In [5]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import torch
import torchaudio

In [6]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

Loading basic things required for whisper translation <br>
Reference: 
HuggingFace:  https://huggingface.co/openai/whisper-large-v3-turbo
Medium:  https://medium.com/@jatin.dhall7385/pythonic-wav-file-handling-a-guide-to-reading-wav-files-without-external-libraries-f5869b27b2e7

In [7]:
# Model 
model_id = "openai/whisper-large-v3-turbo"

In [8]:
# Load the model and processor
model = AutoModelForSpeechSeq2Seq.from_pretrained(model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True)
model.to(device)
processor = AutoProcessor.from_pretrained(model_id)

config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.62G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.77k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.71M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

In [9]:
# Initialize the pipeline
asr_pipeline = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

Device set to use cuda:0


In [10]:
# Sample Audio file path (Example)
file_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train/audio_1024.wav"

In [13]:
# Load audio and preprocess
audio, sr = librosa.load(file_path, sr=16000, mono=True)  # Ensure mono and 16kHz
inputs = processor(audio, sampling_rate=16000, return_tensors="pt").to(device, torch_dtype)

# Generate token ids and decode
with torch.no_grad():
    generated_ids = model.generate(inputs["input_features"])

# Decode the predicted tokens into text
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

print("Transcription:", transcription)

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Transcription:  my favorite plate places is in on the place it is in Chitra district it is it is the it is a temple of see Lord Ventes or a swami it has seven hills to visit the temple it is a good place to


The whisper model can handle input audio in one shot is usually 30 seconds max per segment. (Approx)

So we will perform silding window technique so it captures first 30 sec then followed by remaining time so that in total it captures the entire 60 seconds audio and give us text translation

In [14]:
def transcribe_audio(file_path, segment_length=30):
    # Load audio
    waveform, sr = torchaudio.load(file_path)

    # Convert to mono if stereo
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)

    # Resample to 16kHz if not already
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        waveform = resampler(waveform)

    # Number of samples in each segment
    segment_samples = segment_length * 16000  # 30 sec chunks

    transcriptions = []

    for start in range(0, waveform.size(1), segment_samples):
        end = min(start + segment_samples, waveform.size(1))
        segment = waveform[:, start:end].squeeze().numpy()

        try:
            result = asr_pipeline(segment)
            transcriptions.append(result['text'])
        except Exception as e:
            print(f"Error transcribing segment {start}-{end}: {e}")
            transcriptions.append("")

    return " ".join(transcriptions)

In [12]:
#Loading all library

import os
import pandas as pd
import torch
import numpy as np
import librosa
import time
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

In [15]:
text = transcribe_audio(file_path)
print(text)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


 my favorite plate places is in on the place it is in Chitra district it is it is the it is a temple of see Lord Ventes or a swami it has seven hills to visit the temple it is a good place to  receive the Thirumala for India here crowds of people to visit the Lord Venteshwaraswamy in Thirumala have a lot of a lot of places to visit the visit the tourists in in Thirumala laddu is very famous  you


**Now we had done transcription with respect to few example.
Lets do transcription of entire audios_train to their respective translation in text format and map it to their respective audio file in train_csv**

In [16]:
def load_whisper_model():
    """
    Initializing Whisper Model
    
    Returns:
        pipeline: Hugging Face pipeline for automatic speech recognition

    Reference:
        HuggingFace 
    """
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
    print(f"Using device: {device}")
    
    model_id = "openai/whisper-large-v3-turbo"
    model = AutoModelForSpeechSeq2Seq.from_pretrained(
        model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
    )
    model.to(device)
    
    processor = AutoProcessor.from_pretrained(model_id)
    pipe = pipeline(
        "automatic-speech-recognition",
        model=model,
        tokenizer=processor.tokenizer,
        feature_extractor=processor.feature_extractor,
        torch_dtype=torch_dtype,
        device=device,
    )
    
    return pipe

In [17]:
def load_audio(file_path):
    """
    Load an audio file and convert it to mono if stereo. (As there are some file which are not mono which gave me an error earlier that transcription column doesn't matches the train.csv size)
    
    Args:
        file_path: Path to the audio file
        
    Returns:
        audio_data: Audio data as numpy array
        sample_rate: Sample rate of the audio
     Sample rate refers to the number of audio samples taken per second when digitizing an audio signal. (kHz unit)
    """
    audio, sample_rate = librosa.load(file_path, sr=16000, mono=False)
    
    # Check if audio is stereo (2D array) and convert to mono if needed
    if len(audio.shape) > 1 and audio.shape[0] == 2:
        print(f"Converting stereo to mono for {os.path.basename(file_path)}")
        audio = np.mean(audio, axis=0)
    
    return audio, sample_rate

In [18]:
def transcribe_long_audio(pipe, audio, sample_rate, window_size=30, overlap=1):
    """
    Transcribe audio that may be longer than 30 seconds using a sliding window approach.
    
    Args:
        pipe: Whisper pipeline for transcription (Obtain from load_whisper_model function)
        audio: Audio data as numpy array  (Obtain from load_audio)
        sample_rate: Sample rate of the audio  (Obtain from load_audio)
        window_size: Size of the sliding window in seconds   (Since whisper model can translate 30 sec audio at a time we had used sliding window technique in our work)
        overlap: Overlap between windows in seconds   (Overlapping the time frame between two windows)
        
    Returns:
        full_transcription: Complete transcription of the audio
    """
    # Calculate window and stride sizes in samples
    window_samples = window_size * sample_rate
    stride_samples = (window_size - overlap) * sample_rate
    
    # If audio is shorter than window_size, just transcribe it directly
    if len(audio) <= window_samples:
        result = pipe({"sampling_rate": sample_rate, "raw": audio})
        return result["text"].strip()
    
    # else For longer audio, use sliding window approach
    transcriptions = []
    
    # Calculate number of windows (Though we know that in 2 window we will cover the task but made it more general)
    num_windows = max(1, int(np.ceil((len(audio) - window_samples) / stride_samples)) + 1)
    
    for i in range(num_windows):
        start_sample = int(i * stride_samples)
        end_sample = min(len(audio), start_sample + window_samples)
        
        # Extract audio segment
        audio_segment = audio[start_sample:end_sample]
        
        # Transcribe segment
        result = pipe({"sampling_rate": sample_rate, "raw": audio_segment})
        transcriptions.append(result["text"].strip())
    
    # Merge all transcriptions
    full_transcription = " ".join(transcriptions)
    return full_transcription

**Now we will map all audio translates with their Audio file name in train_csv file & store it in new csv file name train_with_transcription.csv**

In [19]:
def transcribe_audio_files(audio_folder, csv_path):
    """
    Transcribe all audio files listed in the CSV and add transcriptions as a new column.
    
    Args:
        audio_folder: Path to folder containing audio files
        csv_path: Path to CSV file with list of audio files
        
    Returns:
        df: DataFrame with added transcription column
    """

    # Load CSV file
    df = pd.read_csv(csv_path)
    print(f"Loaded CSV with {len(df)} entries")
    
    # Load Whisper model pipeline define earlier
    pipe = load_whisper_model()
    
    # Create a new column for transcriptions
    df['transcription'] = ""
    
    # For loop to iterate through each audio file and add transcription of it
    print("Starting transcription process...")
    for i, row in df.iterrows():
        audio_file = row['filename']  
        full_path = os.path.join(audio_folder, audio_file)
        
        try:
            # Print progress
            print(f"Processing file {i+1}/{len(df)}: {audio_file}")
            
            # Load and convert audio file
            audio, sample_rate = load_audio(full_path)
            
            # Transcribe audio
            transcription = transcribe_long_audio(pipe, audio, sample_rate)
            
            # Add transcription to dataframe
            df.at[i, 'transcription'] = transcription
            
            # Add a small delay to prevent overloading 
            if (i + 1) % 10 == 0:
                print(f"Processed {i+1} files. Taking a short break...")
                time.sleep(1)
                
        except Exception as e:
            print(f"Error processing {audio_file}: {str(e)}")
            df.at[i, 'transcription'] = "ERROR: Could not transcribe"
    
    return df

In [20]:
def main():
    """
    Main function to run the transcription process.
    """
    #  paths
    audio_folder = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_train"
    csv_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv"
    output_path = "/kaggle/working/train_with_transcriptions.csv"
    
    # Transcribe audio files
    df = transcribe_audio_files(audio_folder, csv_path)
    
    # Save the updated dataframe
    df.to_csv(output_path, index=False)
    print(f"Transcription complete! Saved to {output_path}")

In [21]:
if __name__ == "__main__":
    main()

Loaded CSV with 444 entries
Using device: cuda:0


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Starting transcription process...
Processing file 1/444: audio_1261.wav
Processing file 2/444: audio_942.wav
Processing file 3/444: audio_1110.wav
Processing file 4/444: audio_1024.wav


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Processing file 5/444: audio_538.wav
Processing file 6/444: audio_350.wav
Processing file 7/444: audio_64.wav
Processing file 8/444: audio_252.wav
Processing file 9/444: audio_1304.wav
Processing file 10/444: audio_1230.wav
Processed 10 files. Taking a short break...
Processing file 11/444: audio_133.wav
Processing file 12/444: audio_790.wav
Processing file 13/444: audio_947.wav
Processing file 14/444: audio_288.wav
Processing file 15/444: audio_1111.wav
Processing file 16/444: audio_771.wav
Processing file 17/444: audio_1184.wav
Processing file 18/444: audio_918.wav
Processing file 19/444: audio_1030.wav
Processing file 20/444: audio_1112.wav
Processed 20 files. Taking a short break...
Processing file 21/444: audio_873.wav
Processing file 22/444: audio_539.wav
Processing file 23/444: audio_899.wav
Processing file 24/444: audio_1277.wav
Processing file 25/444: audio_649.wav
Processing file 26/444: audio_701.wav
Processing file 27/444: audio_763.wav
Processing file 28/444: audio_952.wav

In [22]:
# # Print some examples
# print("\nExample transcriptions:")
# for i in range(min(5, len(df))):
#     print(f"File: {df.iloc[i]['file_name']}")
#     print(f"Transcription: {df.iloc[i]['transcription']}")
#     print("-" * 50)

In [23]:
import pandas as pd
# Display the first 5 rows of the dataframe with transcriptions
df = pd.read_csv("/kaggle/working/train_with_transcriptions.csv")
df.head()

,filename,label,transcription
0,audio_1261.wav,1.0,Спасибо. Cultivate my favorite hobbies cultiva...
1,audio_942.wav,1.5,the playground looks like very clear and neat ...
2,audio_1110.wav,1.5,my goal is to become an electrical employee an...
3,audio_1024.wav,1.5,my favorite plate places is in on the place it...
4,audio_538.wav,2.0,My favorite place is Uti and Kodai Canal. My e...


In [24]:
# Save the DataFrame to the working directory
output_file_path = "/kaggle/working/transcribed_audio_data.csv"
df.to_csv(output_file_path, index=False)
print(f"Saved transcribed data to {output_file_path}")

Saved transcribed data to /kaggle/working/transcribed_audio_data.csv


In [25]:
df_train = pd.read_csv("/kaggle/input/shl-intern-hiring-assessment/dataset/train.csv")

In [26]:
df_train['label'].min()

1.0

In [27]:
df_train['label'].max()

5.0

In [28]:
# To check that any row has transcription column empty or NULL
df['transcription'].isnull().any() 

False

In [29]:
df_train['label'].value_counts()

label
5.0    110
3.0     87
2.0     70
4.5     58
4.0     52
2.5     40
3.5     23
1.5      3
1.0      1
Name: count, dtype: int64

**Now fine-tuning The DistillBert model for our dataset in regression problem & using PEFT technique(Lora) for training to maintain accuracy with less loss in performance and to utilize memory efficiently & Decrease the model training time.**

In [30]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from datasets import Dataset
from peft import get_peft_model, LoraConfig, TaskType
from sklearn.metrics import mean_squared_error

# Load the transcribed CSV file
df = pd.read_csv("/kaggle/working/transcribed_audio_data.csv")

# Drop the filename column (As we are having )
df_new = df.drop('filename', axis=1)

# Display the dataframe structure
print("DataFrame structure after dropping filename column:")
print(df_new.head())

# Convert to dataset format for Hugging Face 
df_new['label'] = df_new['label'].astype(float)  # Ensure labels are floats

# Use entire dataset for training
train_dataset = Dataset.from_pandas(df_new)

# Load tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define tokenization function
def tokenize_function(examples):
    return tokenizer(examples["transcription"], padding="max_length", truncation=True, max_length=512)

# Tokenize dataset
tokenized_train = train_dataset.map(tokenize_function, batched=True)

# Initialize DistilBERT model for regression
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=1,  # Single output for regression
    problem_type="regression"
)

# Configure LoRA
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,  # Rank
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=["q_lin", "v_lin", "k_lin", "out_lin"]  # DistilBERT attention layers
)

# Apply LoRA to model
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()  # Show trainable vs total parameters

# Fixed custom trainer class with MSE loss
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = nn.MSELoss()
        loss = loss_fct(logits.view(-1), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Training arguments - utilizing both T4 GPUs
training_args = TrainingArguments(
    output_dir="/kaggle/working/distilbert-audio-regression",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=100,
    weight_decay=0.01,
    save_strategy="epoch",
    save_total_limit=1,
    fp16=True,
    report_to="none",
    dataloader_num_workers=2,
    # Add explicit logging settings
    logging_dir="/kaggle/working/logs",
    logging_strategy="steps",
    logging_steps=10,  # Log every 10 steps
    logging_first_step=True,  # Log the first step
)

# Initialize custom trainer with MSE loss
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    tokenizer=tokenizer,
)

# Train model
trainer.train()

# Save the model
trainer.save_model("/kaggle/working/distilbert-audio-regression-final")
print("Model training complete and model saved!")

# Create a simple function to make predictions with the trained model
def predict_rating(text, model, tokenizer):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits.cpu().numpy().squeeze()

# Example prediction function (can be used after training)
print("Prediction function created. You can use predict_rating(text, model, tokenizer) to make predictions.")

DataFrame structure after dropping filename column:
   label                                      transcription
0    1.0  Спасибо. Cultivate my favorite hobbies cultiva...
1    1.5  the playground looks like very clear and neat ...
2    1.5  my goal is to become an electrical employee an...
3    1.5  my favorite plate places is in on the place it...
4    2.0  My favorite place is Uti and Kodai Canal. My e...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

trainable params: 1,181,185 || all params: 68,135,426 || trainable%: 1.7336


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
1,14.388700
10,13.686300
20,11.528700
30,10.645800
40,9.023100
50,7.682900
60,6.038700
70,3.823400
80,2.540200
90,1.475700


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

Model training complete and model saved!
Prediction function created. You can use predict_rating(text, model, tokenizer) to make predictions.


In [31]:
import pandas as pd
import torch
import random
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import PeftModel, PeftConfig

# Load the dataset
df = pd.read_csv("/kaggle/working/train_with_transcriptions.csv")

# Load the saved model and tokenizer
model_path = "/kaggle/working/distilbert-audio-regression-final"
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load the base model and then apply the PEFT adapter
base_model_name = "distilbert-base-uncased"
base_model = AutoModelForSequenceClassification.from_pretrained(
    base_model_name,
    num_labels=1,
    problem_type="regression"
)
model = PeftModel.from_pretrained(base_model, model_path)
model.eval()

# Define prediction function
def predict_rating(text, model, tokenizer):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.logits.cpu().numpy().squeeze()

# Select random samples and make predictions
random.seed(42)  # For reproducibility
num_samples = 5
sample_indices = random.sample(range(len(df)), num_samples)

print(f"\nTesting model on {num_samples} random samples from training data:\n")
print("-" * 80)

for idx in sample_indices:
    sample = df.iloc[idx]
    transcription = sample['transcription']
    true_label = sample['label']
    
    # Get prediction
    predicted_label = predict_rating(transcription, model, tokenizer)
    
    # Calculate error
    error = abs(predicted_label - true_label)
    
    # Print results
    print(f"Sample {idx+1}:")
    print(f"Transcription (truncated): {transcription[:100]}...")
    print(f"True label: {true_label:.2f}")
    print(f"Predicted label: {predicted_label:.2f}")
    print(f"Absolute error: {error:.2f}")
    print("-" * 80)

# Calculate MSE for the random samples
mse_samples = []
for idx in sample_indices:
    sample = df.iloc[idx]
    transcription = sample['transcription']
    true_label = sample['label']
    predicted_label = predict_rating(transcription, model, tokenizer)
    mse_samples.append((predicted_label - true_label) ** 2)

mse = sum(mse_samples) / len(mse_samples)
print(f"Mean Squared Error (MSE) for these samples: {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {mse ** 0.5:.4f}")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



Testing model on 5 random samples from training data:

--------------------------------------------------------------------------------
Sample 328:
Transcription (truncated): The best day of my life is swimming in a cold spring. I'd invention the sensation of cool, clear wat...
True label: 4.50
Predicted label: 3.46
Absolute error: 1.04
--------------------------------------------------------------------------------
Sample 58:
Transcription (truncated): My topic is to describe about the best day in my life and the best day in my life is when I am study...
True label: 2.50
Predicted label: 2.37
Absolute error: 0.13
--------------------------------------------------------------------------------
Sample 13:
Transcription (truncated): Usually a playground looks very huge in a circular or like a bandage curved around. and running etc ...
True label: 2.00
Predicted label: 2.40
Absolute error: 0.40
--------------------------------------------------------------------------------
Sample 380:
T

**Now creating code to add transcription column in test csv file which we will pass to finetuned DistillBert model to predict label**

In [32]:
def main():
    """
    Main function to run the transcription process.
    """
    # Define paths
    audio_folder = "/kaggle/input/shl-intern-hiring-assessment/dataset/audios_test"
    csv_path = "/kaggle/input/shl-intern-hiring-assessment/dataset/test.csv"
    output_path = "/kaggle/working/test_with_transcriptions.csv"
    
    # Transcribe audio files
    df = transcribe_audio_files(audio_folder, csv_path)
    
    # Save the updated dataframe
    df.to_csv(output_path, index=False)
    print(f"Transcription complete! Saved to {output_path}")

In [33]:
if __name__ == "__main__":
    main()

Loaded CSV with 195 entries
Using device: cuda:0


Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:512: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


Starting transcription process...
Processing file 1/195: audio_706.wav
Processing file 2/195: audio_800.wav
Processing file 3/195: audio_68.wav
Processing file 4/195: audio_1267.wav
Processing file 5/195: audio_683.wav
Processing file 6/195: audio_1242.wav
Processing file 7/195: audio_908.wav
Processing file 8/195: audio_888.wav
Processing file 9/195: audio_137.wav
Processing file 10/195: audio_770.wav
Processed 10 files. Taking a short break...
Processing file 11/195: audio_735.wav
Processing file 12/195: audio_1026.wav
Processing file 13/195: audio_1214.wav
Processing file 14/195: audio_1122.wav
Processing file 15/195: audio_1022.wav
Processing file 16/195: audio_726.wav
Processing file 17/195: audio_1205.wav
Processing file 18/195: audio_1116.wav
Processing file 19/195: audio_1240.wav
Processing file 20/195: audio_151.wav
Processed 20 files. Taking a short break...
Processing file 21/195: audio_437.wav
Processing file 22/195: audio_1217.wav
Processing file 23/195: audio_831.wav
Proc

In [34]:
import pandas as pd
# Display the first 5 rows of the dataframe with transcriptions
df_test = pd.read_csv("/kaggle/working/test_with_transcriptions.csv")
df_test.head()

,filename,transcription
0,audio_706.wav,"I love you. I told you, you have to go to the ..."
1,audio_800.wav,My hobbies are playing cricket because I am a ...
2,audio_68.wav,"Yes, at this market you can find a lot of thin..."
3,audio_1267.wav,"My goal is I become an interpreter, entreprene..."
4,audio_683.wav,"Okay, in the cold market there is a lot of peo..."


In [35]:
Test_df = df_test.drop('filename', axis=1)

In [36]:
Test_df.head()

,transcription
0,"I love you. I told you, you have to go to the ..."
1,My hobbies are playing cricket because I am a ...
2,"Yes, at this market you can find a lot of thin..."
3,"My goal is I become an interpreter, entreprene..."
4,"Okay, in the cold market there is a lot of peo..."


**Now we will load our finetuned model n pass our test_with_transcription.csv file**

In [37]:
# Load our fine-tuned model and tokenizer
print("Loading fine-tuned model...")
model_path = "/kaggle/working/distilbert-audio-regression-final"
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained(
    model_path,
    num_labels=1,
    problem_type="regression"
)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()

# Function to predict scores
def predict_rating(text, model, tokenizer, device):
    inputs = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Get the prediction
    prediction = outputs.logits.cpu().numpy().squeeze()
    return prediction

# Make predictions on test data
print("Making predictions...")
predictions = []

for text in df_test['transcription']:
    score = predict_rating(text, model, tokenizer, device)
    predictions.append(score)

# Add predictions to the test dataframe
df_test['label'] = predictions

# Create final submission DataFrame with only required columns
submission_df = pd.DataFrame({
    'filename': df_test['filename'],
    'label': df_test['label']
})

# Save the submission file
submission_df.to_csv("/kaggle/working/submission.csv", index=False)
print("Predictions completed")
print(f"Submission file created with {len(submission_df)} entries")
print("First few predictions:")
print(submission_df.head())

Loading fine-tuned model...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Making predictions...
Predictions completed
Submission file created with 195 entries
First few predictions:
         filename      label
0   audio_706.wav  4.4122214
1   audio_800.wav  2.8147244
2    audio_68.wav  3.1968772
3  audio_1267.wav  2.4250586
4   audio_683.wav  2.8201332


In [38]:
df_submission = pd.read_csv("/kaggle/working/submission.csv")
df_submission.head()

,filename,label
0,audio_706.wav,4.412221
1,audio_800.wav,2.814724
2,audio_68.wav,3.196877
3,audio_1267.wav,2.425059
4,audio_683.wav,2.820133
